In [ ]:
#Imports 
import xarray as xr
import numpy as np
from rioxarray.merge import merge_arrays

# Xarray

## Dataset Operations

In [ ]:
# Now we will create some DataArrays with some fake data, which have a similar structure as eodags single bands.
def DataArray(array, bottom_left:tuple, date:str):
    arr = xr.DataArray(data=array,
                        dims=('band', 'y', 'x'),
                        coords={'x':np.arange(bottom_left[0],bottom_left[0] + array.shape[-1]),
                                'y':np.arange(bottom_left[1],bottom_left[1] + array.shape[1]),
                                'band':[1],
                                'spatial_ref':0},
                            attrs = {'scale_factor':1.0,
                                    'add_offset': 0.0,
                                    '_FillValue': 65535})
    arr = arr.squeeze()
    arr = arr.expand_dims(dim={'time':[date]})
    return arr

#####################
# Tile 1 
#####################

# Time 1, Bands 1,2,3
T1_b1_1, T1_b2_1, T1_b3_1 = [DataArray(np.random.randint((idx+1)*10,(idx+1)*10+10, size=(1,4,5)), bottom_left=(45, 16), date='01.01.2024') for idx in range(3)] 

# Time 2, Bands 1,2,3
T1_b1_2, T1_b2_2, T1_b3_2 = [DataArray(np.random.randint((idx+1)*10,(idx+1)*10+10, size=(1,4,5)), bottom_left=(45, 16), date='02.02.2024') for idx in range(3)] 

#####################
# Tile 2
#####################

# Time 1, Bands 1,2,3
T2_b1_1, T2_b2_1, T2_b3_1 = [DataArray(np.random.randint((idx+1)*10,(idx+1)*10+10, size=(1,4,5)), bottom_left=(41, 16), date='01.01.2024') for idx in range(3)] 

# Tile 2, Time 2, Bands 1,2,3
T2_b1_2, T2_b2_2, T2_b3_2 = [DataArray(np.random.randint((idx+1)*10,(idx+1)*10+10, size=(1,4,5)), bottom_left=(41, 16), date='02.02.2024') for idx in range(3)] 


# Naming the Bands
for elem in [T1_b1_1, T2_b1_1, T1_b1_2, T2_b1_2]:
    elem.name = 'B01'

for elem in [T1_b2_1, T2_b2_1, T1_b2_2, T2_b2_2]:
    elem.name = 'B02'

for elem in [T1_b3_1, T2_b3_1, T1_b3_2, T2_b3_2]:
    elem.name = 'B03'

In [ ]:
# Display one Dataarray
T1_b3_2

In [ ]:
# Load multiple Bands from same Tile and same time.
# With this Method if there are multiple spatial and temporal cordinates, the later ones will overwrite. (See if you add one other Tile)
def Dataset_from_Arrays(arrays):
    load_data = {}
    for elem in arrays:
        load_data[elem.name] = elem
    ds = xr.Dataset(load_data)
    return ds

ds_T1_1 = Dataset_from_Arrays([T1_b1_1, T1_b2_1, T1_b3_1])
ds_T1_2 = Dataset_from_Arrays([T1_b1_2, T1_b2_2, T1_b3_2])

ds_T2_1 = Dataset_from_Arrays([T2_b1_1, T2_b2_1, T2_b3_1])
ds_T2_2 = Dataset_from_Arrays([T2_b1_2, T2_b2_2, T2_b3_2])

In [ ]:
# Merging Datasets with different Timestamps, but same spatial extent
ds_T1 = xr.concat([ds_T1_1, ds_T1_2], dim='time')
ds_T1

In [ ]:
# Merging Datasets with different spatial extent but same timestamps
ds_1 = merge_arrays([arr.squeeze() for arr in [ds_T1_1, ds_T2_1]])
ds_1

## Different Merging Operations

## Reading NetCDF

In [ ]:
path = '../eodag-data/eodag_workspace_post_process\eodag_netCDF\image1.nc'
image = xr.open_dataset(path)
image

In [ ]:
image.Image1

In [ ]:
image.close()